In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 50, bias= True),
            nn.ReLU(),
            nn.Linear(50, 50, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(50, 45, bias = True),
            nn.ReLU(),
            nn.Linear(45, 40, bias= True),
            nn.ReLU(),
            nn.Linear(40, 40, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(40, 30, bias= True),
            nn.ReLU(),
            nn.Linear(30, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Linear(20, 15, bias= True),
            nn.ReLU(),
            nn.Linear(15, 10, bias = True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=50, out_features=45, bias=True)
    (6): ReLU()
    (7): Linear(in_features=45, out_features=40, bias=True)
    (8): ReLU()
    (9): Linear(in_features=40, out_features=40, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=40, out_features=30, bias=True)
    (13): ReLU()
    (14): Linear(in_features=30, out_features=25, bias=True)
    (15): ReLU()
    (16): Linear(in_features=25, out_features=20, bias=True)
    (17): ReLU()
    (18): Linear(in_features=20, out_features=15, bias=True)
    (19): ReLU()
    (20): Linear(in_features=15, out_features=10, bias=True)
    (21): ReLU()
    (22): Linear(in_features=10, out_features=5, bias=True)
    (23): ReLU()
  )
  (classify): Sequential(
    (0):

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [ ]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3327.011797785759
loss - 3315.528683960438
loss - 3313.1963059306145
loss - 3309.9206636548042
loss - 3305.7126828432083
loss - 3300.873572945595
loss - 3298.735218465328
loss - 3297.0859273076057
loss - 3295.5609126091003
loss - 3294.936773478985
loss - 3294.1416498422623
loss - 3292.83951741457
loss - 3291.76499325037
loss - 3290.712089717388
loss - 3289.3188887238503
loss - 3288.6895500421524
loss - 3286.4146760702133
loss - 3286.4320939183235
loss - 3283.735194683075
loss - 3281.6708720326424
loss - 3278.817104458809
loss - 3276.030276298523
loss - 3272.5797830224037
loss - 3270.8911404013634
loss - 3269.3712481856346
loss - 3269.5003615021706
loss - 3266.8577151298523
loss - 3266.9860305786133
loss - 3265.5582768917084
loss - 3264.7908462285995
loss - 3263.9965721964836
loss - 3262.673334658146
loss - 3261.7048082351685
loss - 3260.6684809327126
loss - 3259.6910140514374
loss - 3258.5579754710197
loss - 3257.740830242634
loss - 3256.8678206801414
loss - 3254.4135489463806
l

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6132886675504281
